In [3]:
import numpy as np
import pandas as pd

In [4]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tmdb 5000 project/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tmdb 5000 project/tmdb_5000_credits.csv')

In [5]:
movies = movies.merge(credits, on = 'title')

In [6]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [7]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
movies.dropna(inplace=True)

In [10]:
import ast          #search ast and literal_eval
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [11]:
movies['genres'] = movies['genres'].apply(convert)

In [12]:
movies['keywords'] = movies['keywords'].apply(convert)

In [13]:
def convert3(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter += 1
    else:
      break
  return L

In [14]:
movies['cast'] = movies['cast'].apply(convert3)

In [15]:
def fetch_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
      break
  return L

In [16]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [17]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [18]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "")for i in x])

In [19]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "")for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "")for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "")for i in x])

In [20]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']

In [21]:
new_df = movies[['movie_id', 'title', 'tags']]

In [22]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-22-ad44e9ca1347>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [23]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

<ipython-input-23-8b60b591a07f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [24]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [25]:
#stemming
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [26]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-26-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [28]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
similarity = cosine_similarity(vectors)

In [31]:
# def recommend(movie):
#     index = new_df[new_df['title'] == movie].index[0]
#     distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     for i in distances[1:6]:
#         print(new_df.iloc[i[0]].title)


In [32]:
def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [34]:
import pickle

In [35]:
pickle.dump(new_df,open('movies_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))